# AI 硬件算力演进分析
## 基于 Epoch AI Machine Learning Hardware 数据集

---

**作者**: 肖景铭  
**学校**: 哈尔滨工业大学  
**学号**: 2023112881  
**课程**: Python程序设计  
**日期**: 2025-12-26

---

### 📋 项目简介

本项目基于 **Epoch AI** 公开发布的 **Machine Learning Hardware 数据集**（170+ GPU/TPU 规格数据），对 **2007-2025 年 AI 硬件算力演进**进行系统性分析。

**为什么要分析 AI 硬件？**

- 📊 **项目驱动**：项目需求，最近在做姿态提取的CV项目，在考虑租借高性能算力服务器
- 🤖 **AI 大模型的崛起**：ChatGPT/GPT-5/Llama 等大模型的训练与推理都依赖强大的硬件算力
- 💰 **数据中心成本**：算力、功耗、显存是数据中心选型的核心指标
- 📈 **硬件创新速度**：AI 芯片的迭代速度远超传统 CPU（在深度学习推理上能快30-50倍），了解趋势有助于预测未来

---

### ❓ 核心研究问题

本项目围绕"**AI 推理硬件的算力演进与选型建议**"展开，重点关注 **INT8/FP8 推理能力**（因为推理场景更贴近实际应用部署）。

#### RQ0: 数据理解与口径统一
- 不同数值格式（FP32/FP16/FP8/INT8）分别用于什么场景？
- 为什么 INT8 算力比 FP32 高出数倍？

#### RQ1: 算力随时间的演进
- 2012-2025 年间，推理算力（INT8）增长了多少倍？
- NVIDIA/AMD/Google/Intel/AWS 谁在不同阶段领先？

#### RQ2: 能效与功耗约束
- 在 TDP 限制下，谁的推理能效（TOP/s/W）最高？

#### RQ3: 显存/带宽与算力的"供给匹配"
- 算力增长是否伴随显存/带宽同步增长？

#### RQ4: 价格与性价比（限有价格硬件）
- 消费级 GPU 的推理性价比如何？

#### RQ5: 结论与选型建议
- 训练/推理两类场景分别推荐哪些硬件？

---

### 📚 通俗解释：什么是 FP32/FP16/INT8？

AI 硬件的算力通常用不同"精度"（数值格式）来衡量，就像视频画质一样：

| 精度 | 适用场景 | 通俗类比 |
|------|----------|----------|
| **FP32** | 传统深度学习训练/推理 | 相当于"标准清晰度"，兼容性最好但速度较慢 |
| **FP16/BF16** | 现代训练主流（混合精度） | "高清"画质，速度快且精度损失可接受 |
| **FP8** | 新一代推理/训练 | "超高清 HDR"，速度更快，新硬件才支持 |
| **INT8** | 推理部署（量化后模型） | "高效压缩格式"，速度极快但需要精心调优 |

**为什么低精度算力更高？**
- 低精度计算需要的电路更少 → 同一个芯片可以并行更多计算单元
- 因此 INT8 算力常常是 FP32 的 **4-16 倍**

---

**让我们开始分析吧！** 🚀

---

## Block 01 - 环境设置与导入模块

**目标**: 导入必要的库，初始化可视化主题，确保输出目录存在


In [ ]:

# ============================================================
# Block 01 - 环境设置与导入模块
# 目标：导入库，初始化配置，创建输出目录
# ============================================================

import sys
import warnings
import os
import platform
import csv
import logging
from pathlib import Path
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

warnings.filterwarnings('ignore')

# ============================================================
# 配置参数（内联，不依赖外部文件）
# ============================================================

# 路径配置
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT
OUTPUT_DIR = PROJECT_ROOT / "output"
VIZ_DIR = PROJECT_ROOT / "visualization"
ML_HARDWARE_CSV = DATA_DIR / "ml_hardware.csv"
OUTPUT_TABLES_DIR = OUTPUT_DIR / "tables"
OUTPUT_DERIVED_DIR = OUTPUT_DIR / "derived"
OUTPUT_LOGS_DIR = OUTPUT_DIR / "logs"

# 可视化子目录
VIZ_OVERVIEW_DIR = VIZ_DIR / "00_dataset_overview"
VIZ_PERF_TRENDS_DIR = VIZ_DIR / "01_perf_trends"
VIZ_EFFICIENCY_DIR = VIZ_DIR / "02_efficiency"
VIZ_MEMORY_DIR = VIZ_DIR / "03_memory_bandwidth"
VIZ_PRICE_DIR = VIZ_DIR / "04_price_value"
VIZ_APPENDIX_DIR = VIZ_DIR / "99_appendix"

# 分析参数
YEAR_RANGE = (2012, 2025)
TRAIN_METRIC = "Tensor-FP16/BF16 performance (FLOP/s)"
INFERENCE_METRIC = "INT8 performance (OP/s)"
FP32_METRIC = "FP32 (single precision) performance (FLOP/s)"
FP8_METRIC = "FP8 performance (FLOP/s)"
TOP_N = 20
MISSING_THRESHOLD = 0.5

# 可视化配置
FIGSIZE = (12, 8)
DPI = 300
FONT_SIZE = 11
TITLE_SIZE = 14
LABEL_SIZE = 12
LEGEND_SIZE = 10
GRID_ALPHA = 0.3
SNS_STYLE = "whitegrid"
SNS_PALETTE = "tab10"
SNS_CONTEXT = "notebook"

# 单位换算常数
TERA = 1e12
PETA = 1e15
KB = 1024
MB = KB ** 2
GB = KB ** 3
TB = KB ** 4
GB_DECIMAL = 1e9
TB_DECIMAL = 1e12

# 厂商颜色映射
MANUFACTURER_COLORS = {
    "NVIDIA": "#76B900",
    "AMD": "#ED1C24",
    "Google": "#4285F4",
    "Intel": "#0071C5",
    "Amazon AWS": "#FF9900",
    "Meta": "#0668E1",
}

# 数值字段列表
NUMERIC_COLS = [
    "TDP (W)",
    "FP64 (double precision) performance (FLOP/s)",
    "FP32 (single precision) performance (FLOP/s)",
    "TF32 (TensorFloat-32) performance (FLOP/s)",
    "FP16 (half precision) performance (FLOP/s)",
    "Tensor-FP16/BF16 performance (FLOP/s)",
    "FP8 performance (FLOP/s)",
    "FP4 performance (FLOP/s)",
    "INT16 performance (OP/s)",
    "INT8 performance (OP/s)",
    "INT4 performance (OP/s)",
    "Memory (bytes)",
    "Memory bandwidth (byte/s)",
    "Intranode bandwidth (byte/s)",
    "Internode bandwidth (bit/s)",
    "Release price (USD)",
    "Energy efficiency",
    "Max performance",
    "Total processing performance (bit-OP/s)",
    "Price-performance",
    "ML OP/s",
]

# ============================================================
# 工具函数（内联）
# ============================================================

def ensure_dirs():
    """确保所有输出目录存在"""
    dirs = [
        OUTPUT_DIR, OUTPUT_TABLES_DIR, OUTPUT_DERIVED_DIR, OUTPUT_LOGS_DIR,
        VIZ_DIR, VIZ_OVERVIEW_DIR, VIZ_PERF_TRENDS_DIR, VIZ_EFFICIENCY_DIR,
        VIZ_MEMORY_DIR, VIZ_PRICE_DIR, VIZ_APPENDIX_DIR
    ]
    for d in dirs:
        d.mkdir(parents=True, exist_ok=True)

def setup_logging():
    """配置日志系统"""
    ensure_dirs()
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(OUTPUT_LOGS_DIR / "analysis.log", encoding='utf-8'),
            logging.StreamHandler()
        ]
    )
    logging.info("日志系统初始化完成")

def get_manufacturer_color(manufacturer):
    """获取厂商对应的颜色"""
    return MANUFACTURER_COLORS.get(manufacturer, "#808080")

def init_viz_theme():
    """初始化可视化主题（包含中文字体配置）"""
    sns.set_style(SNS_STYLE)
    sns.set_context(SNS_CONTEXT)
    sns.set_palette(SNS_PALETTE)
    
    plt.rcParams.update({
        'figure.dpi': DPI,
        'savefig.dpi': DPI,
        'font.size': FONT_SIZE,
        'axes.titlesize': TITLE_SIZE,
        'axes.labelsize': LABEL_SIZE,
        'xtick.labelsize': FONT_SIZE,
        'ytick.labelsize': FONT_SIZE,
        'legend.fontsize': LEGEND_SIZE,
        'figure.figsize': FIGSIZE,
        'axes.grid': True,
        'grid.alpha': GRID_ALPHA,
        'figure.autolayout': True,
    })
    
    # 跨平台中文字体配置
    SYSTEM = platform.system()
    if SYSTEM == 'Windows':
        FONT_SANS_SERIF = ['SimHei']
    elif SYSTEM == 'Darwin':
        FONT_SANS_SERIF = ['PingFang SC', 'Arial Unicode MS']
    elif SYSTEM == 'Linux':
        FONT_SANS_SERIF = ['WenQuanYi Micro Hei', 'Noto Sans CJK SC']
    else:
        FONT_SANS_SERIF = ['DejaVu Sans']
    
    # 检测并设置可用的中文字体
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    font_candidates = FONT_SANS_SERIF + ['Microsoft YaHei', 'SimSun', 'KaiTi', 'FangSong']
    
    selected_font = None
    for font in font_candidates:
        if font in available_fonts:
            selected_font = font
            break
    
    if selected_font:
        plt.rcParams['font.sans-serif'] = [selected_font] + [f for f in FONT_SANS_SERIF if f != selected_font]
        logging.info(f"✅ 找到中文字体: {selected_font}")
    else:
        plt.rcParams['font.sans-serif'] = FONT_SANS_SERIF
        logging.warning(f"⚠️ 未找到中文字体，使用配置: {FONT_SANS_SERIF}")
    
    plt.rcParams['font.family'] = ['sans-serif']
    plt.rcParams['axes.unicode_minus'] = False
    logging.info("✅ 可视化主题初始化完成（已配置中文字体）")

# ============================================================
# 初始化
# ============================================================

ensure_dirs()
setup_logging()
init_viz_theme()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ 环境设置完成！")
print(f"📁 项目根目录: {PROJECT_ROOT}")
print(f"📊 数据文件: {ML_HARDWARE_CSV}")
print(f"📈 可视化目录: {VIZ_DIR}")
print(f"\n⚙️  分析配置:")
print(f"   - 时间范围: {YEAR_RANGE}")
print(f"   - 推理口径: {INFERENCE_METRIC}")
print(f"   - Top-N: {TOP_N}")
print(f"\n🎨 中文字体: {plt.rcParams['font.sans-serif']}")

2025-12-26 21:02:40,157 - INFO - 日志系统初始化完成
2025-12-26 21:02:40,159 - INFO - ✅ 可视化主题初始化完成（已配置中文字体）


✅ 环境设置完成！
📁 项目根目录: c:\Users\jimmy\Desktop\python程序设计\作业4
📊 数据文件: c:\Users\jimmy\Desktop\python程序设计\作业4\ml_hardware.csv
📈 可视化目录: c:\Users\jimmy\Desktop\python程序设计\作业4\visualization

⚙️  分析配置:
   - 时间范围: (2012, 2025)
   - 推理口径: INT8 performance (OP/s)
   - Top-N: 20

🎨 中文字体: ['Arial', 'DejaVu Sans', 'Liberation Sans', 'Bitstream Vera Sans', 'sans-serif']


## Block 02 - 数据读取与工具函数

**目标**: 定义数据读取、清洗、保存等通用函数

In [ ]:

# ============================================================
# 数据读取与处理函数
# ============================================================

def load_ml_hardware_data(filepath=ML_HARDWARE_CSV, encoding='utf-8'):
    """读取 ML Hardware 数据集（处理 Notes 多行/引号问题）"""
    logging.info(f"正在读取数据: {filepath}")
    
    try:
        df = pd.read_csv(
            filepath,
            encoding=encoding,
            low_memory=False,
            na_values=['', 'NA', 'N/A', 'nan', 'NaN', 'null']
        )
        logging.info(f"✅ 数据读取成功: {df.shape[0]} 行 × {df.shape[1]} 列")
        return df
    except Exception as e:
        logging.warning(f"⚠️ C engine 解析失败: {e}")
        logging.info("尝试使用 Python engine 回退...")
        try:
            df = pd.read_csv(
                filepath,
                encoding=encoding,
                engine='python',
                quoting=csv.QUOTE_MINIMAL,
                escapechar='\\',
                na_values=['', 'NA', 'N/A', 'nan', 'NaN', 'null']
            )
            logging.info(f"✅ 数据读取成功（Python engine）: {df.shape[0]} 行 × {df.shape[1]} 列")
            return df
        except Exception as e2:
            logging.error(f"❌ 数据读取失败: {e2}")
            raise

def parse_release_date(df, date_col='Release date'):
    """解析发布日期（字符串 → datetime）"""
    logging.info(f"解析日期字段: {date_col}")
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df['release_year'] = df[date_col].dt.year
    df['release_month'] = df[date_col].dt.month
    missing_count = df[date_col].isna().sum()
    if missing_count > 0:
        logging.warning(f"⚠️ {date_col} 缺失 {missing_count} 条记录")
    return df

def convert_numeric_cols(df, cols=None):
    """将指定列转为数值类型"""
    if cols is None:
        cols = NUMERIC_COLS
    logging.info(f"转换数值列: {len(cols)} 个字段")
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def add_derived_metrics(df):
    """添加派生指标（单位换算 + 能效 + 比率）"""
    logging.info("计算派生指标...")
    
    # 算力换算（FLOP/s → TFLOP/s, OP/s → TOP/s）
    if FP32_METRIC in df.columns:
        df['perf_fp32_tflops'] = df[FP32_METRIC] / TERA
    if TRAIN_METRIC in df.columns:
        df['perf_fp16_tflops'] = df[TRAIN_METRIC] / TERA
    if FP8_METRIC in df.columns:
        df['perf_fp8_tflops'] = df[FP8_METRIC] / TERA
    if INFERENCE_METRIC in df.columns:
        df['int8_tops'] = df[INFERENCE_METRIC] / TERA
    
    # 显存换算（bytes → GB）
    if 'Memory (bytes)' in df.columns:
        df['mem_gb'] = df['Memory (bytes)'] / GB
    
    # 带宽换算（byte/s → TB/s）
    if 'Memory bandwidth (byte/s)' in df.columns:
        df['mem_bw_tbs'] = df['Memory bandwidth (byte/s)'] / TB_DECIMAL
    
    # 能效（算力 / TDP）
    if INFERENCE_METRIC in df.columns and 'TDP (W)' in df.columns:
        df['efficiency_int8_per_w'] = df[INFERENCE_METRIC] / df['TDP (W)']
    if TRAIN_METRIC in df.columns and 'TDP (W)' in df.columns:
        df['efficiency_fp16_per_w'] = df[TRAIN_METRIC] / df['TDP (W)']
    
    logging.info("✅ 派生指标计算完成")
    return df

def get_missing_summary(df):
    """生成缺失值汇总表"""
    missing_count = df.isnull().sum()
    missing_rate = (missing_count / len(df) * 100).round(2)
    non_missing = len(df) - missing_count
    
    summary = pd.DataFrame({
        'Field': df.columns,
        'Missing Count': missing_count.values,
        'Missing Rate (%)': missing_rate.values,
        'Non-Missing': non_missing.values
    })
    summary = summary.sort_values('Missing Rate (%)', ascending=False)
    summary = summary[summary['Missing Count'] > 0]
    return summary

def check_duplicates(df, subset=['Hardware name', 'Release date']):
    """检查重复记录"""
    duplicates = df[df.duplicated(subset=subset, keep=False)]
    if len(duplicates) > 0:
        logging.warning(f"⚠️ 发现 {len(duplicates)} 条重复记录")
    else:
        logging.info("✅ 无重复记录")
    return duplicates

def filter_by_year_range(df, year_col='release_year', year_range=YEAR_RANGE):
    """按年份范围筛选数据"""
    df_filtered = df[
        (df[year_col] >= year_range[0]) &
        (df[year_col] <= year_range[1])
    ].copy()
    logging.info(f"按年份筛选 ({year_range[0]}-{year_range[1]}): {len(df)} → {len(df_filtered)} 条记录")
    return df_filtered

def get_top_n(df, metric, n=TOP_N, ascending=False):
    """获取 Top-N 记录"""
    df_top = df.nlargest(n, metric) if not ascending else df.nsmallest(n, metric)
    return df_top

def save_table(df, filename, subdir='tables', index=False):
    """保存表格到 output/tables/ 或指定子目录"""
    ensure_dirs()
    if subdir == 'tables':
        out_dir = OUTPUT_TABLES_DIR
    elif subdir == 'derived':
        out_dir = OUTPUT_DERIVED_DIR
    else:
        out_dir = OUTPUT_DIR / subdir
        out_dir.mkdir(parents=True, exist_ok=True)
    
    filepath = out_dir / filename
    if filepath.suffix == '.csv':
        df.to_csv(filepath, index=index, encoding='utf-8-sig')
    elif filepath.suffix in ['.xlsx', '.xls']:
        df.to_excel(filepath, index=index)
    else:
        filepath = filepath.with_suffix('.csv')
        df.to_csv(filepath, index=index, encoding='utf-8-sig')
    logging.info(f"✅ 表格已保存: {filepath}")

print("✅ 数据读取与工具函数定义完成")


## Block 03 - 可视化函数库

**目标**: 定义统一风格的可视化函数（散点图、条形图、气泡图等）

In [ ]:
# ============================================================
# 可视化函数（内联）
# ============================================================

def save_fig(fig, filename, subdir='00_dataset_overview', tight=True, transparent=False):
    """保存图表到 visualization/ 指定子目录"""
    ensure_dirs()
    subdir_map = {
        '00_dataset_overview': VIZ_OVERVIEW_DIR,
        '01_perf_trends': VIZ_PERF_TRENDS_DIR,
        '02_efficiency': VIZ_EFFICIENCY_DIR,
        '03_memory_bandwidth': VIZ_MEMORY_DIR,
        '04_price_value': VIZ_PRICE_DIR,
        '99_appendix': VIZ_APPENDIX_DIR,
    }
    out_dir = subdir_map.get(subdir, VIZ_DIR / subdir)
    out_dir.mkdir(parents=True, exist_ok=True)
    filepath = out_dir / filename
    if tight:
        fig.tight_layout()
    fig.savefig(filepath, dpi=DPI, transparent=transparent, bbox_inches='tight')
    logging.info(f"✅ 图表已保存: {filepath}")
    plt.close(fig)

def plot_missing_heatmap(df, title="Missing Rate Heatmap", filename="missing_rate_heatmap.png", min_missing_rate=50.0):
    """绘制缺失率热力图"""
    missing_rate = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
    missing_rate = missing_rate[missing_rate >= min_missing_rate]
    if len(missing_rate) == 0:
        logging.warning(f"⚠️ 无缺失率 >= {min_missing_rate}% 的字段，跳过热力图")
        return None
    fig, ax = plt.subplots(figsize=(10, max(6, len(missing_rate) * 0.3)))
    sns.heatmap(
        missing_rate.values.reshape(-1, 1),
        annot=True, fmt='.1f', cmap='YlOrRd',
        cbar_kws={'label': 'Missing Rate (%)'},
        yticklabels=missing_rate.index, xticklabels=[''], ax=ax
    )
    ax.set_title(f"{title}\n(仅显示缺失率 ≥ {min_missing_rate}% 的字段)", fontsize=TITLE_SIZE, fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel('Field', fontsize=LABEL_SIZE)
    save_fig(fig, filename, subdir='00_dataset_overview')
    return fig

def plot_barh(data, x_col, y_col, title, xlabel, ylabel, filename, subdir='00_dataset_overview', color=None, top_n=None):
    """绘制横向条形图（适合排名/Top-N）"""
    if top_n:
        data = data.nlargest(top_n, x_col)
    fig, ax = plt.subplots(figsize=(12, max(6, len(data) * 0.4)))
    ax.barh(data[y_col], data[x_col], color=color or sns.color_palette()[0])
    ax.set_title(title, fontsize=TITLE_SIZE, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=LABEL_SIZE)
    ax.set_ylabel(ylabel, fontsize=LABEL_SIZE)
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=GRID_ALPHA)
    save_fig(fig, filename, subdir=subdir)
    return fig

def plot_scatter_trend(df, x_col, y_col, hue_col=None, title="", xlabel="", ylabel="", 
                       filename="scatter_trend.png", subdir='01_perf_trends', 
                       log_y=False, log_x=False, annotate_top_n=10, annotate_col='Hardware name'):
    """绘制散点图 + 趋势（适合时间序列/相关分析）"""
    if hue_col:
        df_plot = df[[x_col, y_col, hue_col, annotate_col]].dropna(subset=[x_col, y_col])
    else:
        df_plot = df[[x_col, y_col, annotate_col]].dropna(subset=[x_col, y_col])
    
    if len(df_plot) == 0:
        logging.warning(f"⚠️ {title}: 无有效数据，跳过绘图")
        return None
    
    fig, ax = plt.subplots(figsize=FIGSIZE)
    
    # 散点图
    if hue_col:
        for label, group in df_plot.groupby(hue_col):
            color = get_manufacturer_color(label) if hue_col == 'Manufacturer' else None
            ax.scatter(group[x_col], group[y_col], label=label, alpha=0.7, s=80, color=color)
    else:
        ax.scatter(df_plot[x_col], df_plot[y_col], alpha=0.7, s=80, color=sns.color_palette()[0])
    
    # 对数轴
    if log_y:
        ax.set_yscale('log')
    if log_x:
        ax.set_xscale('log')
    
    # 标注 Top-N 点（使用 adjustText 避免重叠）
    if annotate_top_n and annotate_col:
        top_data = df_plot.nlargest(annotate_top_n, y_col)
        try:
            from adjustText import adjust_text
            texts = []
            for _, row in top_data.iterrows():
                text = ax.annotate(
                    row[annotate_col],
                    (row[x_col], row[y_col]),
                    fontsize=9, fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.4', facecolor='white', edgecolor='gray', alpha=0.8, linewidth=1),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2', color='gray', alpha=0.6, lw=1)
                )
                texts.append(text)
            adjust_text(texts, ax=ax, expand_points=(1.2, 1.2), expand_text=(1.2, 1.2),
                       force_points=(0.3, 0.3), force_text=(0.3, 0.3),
                       arrowprops=dict(arrowstyle='->', color='gray', lw=0.5))
        except ImportError:
            logging.warning("⚠️ adjustText 未安装，使用基础标注（可能重叠）")
            for i, (_, row) in enumerate(top_data.iterrows()):
                offset_x = 10 if i % 2 == 0 else -10
                offset_y = 10 if i % 3 == 0 else -10
                ax.annotate(
                    row[annotate_col],
                    (row[x_col], row[y_col]),
                    xytext=(offset_x, offset_y), textcoords='offset points',
                    fontsize=9, fontweight='bold',
                    bbox=dict(boxstyle='round,pad=0.4', facecolor='white', edgecolor='gray', alpha=0.8, linewidth=1),
                    arrowprops=dict(arrowstyle='->', color='gray', alpha=0.6, lw=1)
                )
    
    ax.set_title(title, fontsize=TITLE_SIZE, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=LABEL_SIZE)
    ax.set_ylabel(ylabel, fontsize=LABEL_SIZE)
    if hue_col:
        ax.legend(loc='best', fontsize=LEGEND_SIZE, framealpha=0.9)
    ax.grid(True, alpha=GRID_ALPHA)
    save_fig(fig, filename, subdir=subdir)
    return fig

def plot_bubble(df, x_col, y_col, size_col, hue_col, title, xlabel, ylabel, 
                filename, subdir='03_memory_bandwidth', log_x=False, log_y=False, size_scale=200):
    """绘制气泡图（x/y/size/color 四维展示）"""
    df_plot = df[[x_col, y_col, size_col, hue_col]].dropna()
    if len(df_plot) == 0:
        logging.warning(f"⚠️ {title}: 无有效数据，跳过绘图")
        return None
    fig, ax = plt.subplots(figsize=FIGSIZE)
    for label, group in df_plot.groupby(hue_col):
        color = get_manufacturer_color(label) if hue_col == 'Manufacturer' else None
        ax.scatter(group[x_col], group[y_col], s=group[size_col] * size_scale,
                   label=label, alpha=0.6, color=color, edgecolors='black', linewidth=0.5)
    if log_x:
        ax.set_xscale('log')
    if log_y:
        ax.set_yscale('log')
    ax.set_title(title, fontsize=TITLE_SIZE, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=LABEL_SIZE)
    ax.set_ylabel(ylabel, fontsize=LABEL_SIZE)
    ax.legend(loc='best', fontsize=LEGEND_SIZE, framealpha=0.9, scatterpoints=1)
    ax.grid(True, alpha=GRID_ALPHA)
    save_fig(fig, filename, subdir=subdir)
    return fig

def plot_distribution(df, col, title, xlabel, filename, subdir='00_dataset_overview', bins=30, log_scale=False):
    """绘制分布图（直方图 + KDE）"""
    data = df[col].dropna()
    if len(data) == 0:
        logging.warning(f"⚠️ {title}: 无有效数据，跳过绘图")
        return None
    fig, ax = plt.subplots(figsize=FIGSIZE)
    sns.histplot(data, bins=bins, kde=True, ax=ax, color=sns.color_palette()[0], alpha=0.6)
    if log_scale:
        ax.set_xscale('log')
    ax.set_title(title, fontsize=TITLE_SIZE, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=LABEL_SIZE)
    ax.set_ylabel('Frequency', fontsize=LABEL_SIZE)
    ax.grid(True, alpha=GRID_ALPHA)
    save_fig(fig, filename, subdir=subdir)
    return fig

def plot_facet_trend(df, x_col, y_col, facet_col, title, xlabel, ylabel, filename, subdir='01_perf_trends', log_y=False):
    """绘制分面图（按 facet_col 分组，每组一个子图）"""
    df_plot = df[[x_col, y_col, facet_col]].dropna()
    if len(df_plot) == 0:
        logging.warning(f"⚠️ {title}: 无有效数据，跳过绘图")
        return None
    g = sns.FacetGrid(df_plot, col=facet_col, col_wrap=3, height=4, aspect=1.2, sharey=False)
    g.map(sns.scatterplot, x_col, y_col, alpha=0.7, s=50)
    if log_y:
        for ax in g.axes.flat:
            ax.set_yscale('log')
    g.set_titles(col_template="{col_name}", size=TITLE_SIZE)
    g.set_axis_labels(xlabel, ylabel, fontsize=LABEL_SIZE)
    g.fig.suptitle(title, fontsize=TITLE_SIZE + 2, fontweight='bold', y=1.02)
    save_fig(g.fig, filename, subdir=subdir)
    return g.fig

print("✅ 可视化函数定义完成")



## Block 04 - 数据读取与初步探索

**目标**: 读取 CSV，查看基本信息、缺失值

In [ ]:

# ============================================================
# 数据读取与初步探索
# ============================================================

df_raw = load_ml_hardware_data()

print("\n" + "="*60)
print("📊 数据集基本信息")
print("="*60)
print(f"行数: {len(df_raw)}")
print(f"列数: {df_raw.shape[1]}")
print(f"\n前 5 列名: {list(df_raw.columns[:5])}")

# 缺失值统计
missing_summary = get_missing_summary(df_raw)
print("\n⚠️  缺失值 Top 5:")
print(missing_summary.head(5))
save_table(missing_summary, 'missing_value_summary.csv')

print("\n📌 初步发现:")
print("- 价格字段缺失严重（~80%），数据中心硬件不零售")
print("- 算力字段完整度较高（~95%）")
print("- 带宽/互联字段中度缺失（~50%）")


## Block 05 - 数据清洗与派生指标

**目标**: 解析日期、转换数值、计算 TFLOP/s、能效等

In [ ]:
# ============================================================
# 数据清洗与派生指标
# ============================================================

df = df_raw.copy()

# 清洗流程
df = parse_release_date(df)
df = convert_numeric_cols(df)
df = add_derived_metrics(df)
df = filter_by_year_range(df, year_range=YEAR_RANGE)

# 去重
duplicates = check_duplicates(df)
if len(duplicates) > 0:
    print(f"\n⚠️  发现 {len(duplicates)} 条重复，已去重")
    df = df.drop_duplicates(subset=['Hardware name', 'Release date'], keep='first')

print(f"\n✅ 清洗完成：{len(df)} 行 × {df.shape[1]} 列")
print(f"\n新增派生字段:")
for col in ['int8_tops', 'mem_gb', 'efficiency_int8_per_w']:
    if col in df.columns:
        print(f"   - {col}: {df[col].notna().sum()} 非空")

save_table(df, 'cleaned_data_with_derived.csv', subdir='derived')



## Block 06 - 数据概览可视化

**目标**: 缺失率热图、厂商分布、类型分布

In [ ]:

# ============================================================
# 数据概览可视化
# ============================================================

# 缺失率热图（只显示缺失率 >= 50% 的字段）
plot_missing_heatmap(df, title="数据集字段缺失率 (2012-2025)", filename="missing_rate_heatmap.png", min_missing_rate=50.0)

# 厂商分布
manufacturer_counts = df['Manufacturer'].value_counts().reset_index()
manufacturer_counts.columns = ['Manufacturer', 'Count']
plot_barh(manufacturer_counts, x_col='Count', y_col='Manufacturer',
          title="AI 硬件厂商分布 (2012-2025)", xlabel="数量", ylabel="厂商",
          filename="manufacturer_distribution.png", subdir='00_dataset_overview')

# 类型分布（改用条形图，避免饼图重叠）
type_counts = df['Type'].value_counts().reset_index()
type_counts.columns = ['Type', 'Count']
type_counts['Percentage'] = (type_counts['Count'] / type_counts['Count'].sum() * 100).round(1)

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(type_counts['Type'], type_counts['Count'], 
               color=sns.color_palette('Set2', len(type_counts)))

# 在条形上添加数值和百分比标签
for i, (idx, row) in enumerate(type_counts.iterrows()):
    count = row['Count']
    pct = row['Percentage']
    ax.text(count + max(type_counts['Count']) * 0.01, i, 
            f'{count} ({pct}%)', 
            va='center', fontsize=10, fontweight='bold')

ax.set_title("硬件类型分布 (GPU vs TPU)", fontsize=TITLE_SIZE, fontweight='bold')
ax.set_xlabel("数量 (Count)", fontsize=LABEL_SIZE)
ax.set_ylabel("硬件类型 (Type)", fontsize=LABEL_SIZE)
ax.invert_yaxis()
ax.grid(axis='x', alpha=GRID_ALPHA)
save_fig(fig, "type_distribution.png", subdir='00_dataset_overview')

print("✅ 数据概览图已保存")


## Block 07 - RQ1: INT8 推理算力演进

**目标**: 时间序列散点图（log 轴）+ Top 20 排行

In [ ]:
# ============================================================
# RQ1: INT8 推理算力演进分析
# ============================================================

print("\n" + "="*60)
print("📊 RQ1: INT8 推理算力演进分析")
print("="*60)

df_int8 = df[df['int8_tops'].notna()].copy()
print(f"有 INT8 数据: {len(df_int8)} / {len(df)} ({len(df_int8)/len(df)*100:.1f}%)")

# 趋势图（增加标注数量，改进样式）
plot_scatter_trend(
    df_int8, x_col='release_year', y_col='int8_tops', hue_col='Manufacturer',
    title="INT8 推理算力演进 (2012-2025)\n对数轴展示指数增长",
    xlabel="发布年份", ylabel="INT8 算力 (TOP/s, log)",
    filename="int8_performance_over_time.png", subdir='01_perf_trends',
    log_y=True, annotate_top_n=15, annotate_col='Hardware name'
)

# 增长统计
df_int8_sorted = df_int8.sort_values('release_year')
earliest = df_int8_sorted.iloc[0]
latest = df_int8_sorted.iloc[-1]
growth = latest['int8_tops'] / earliest['int8_tops']
years = latest['release_year'] - earliest['release_year']
cagr = (growth ** (1/years) - 1) * 100

print(f"\n📈 INT8 算力增长:")
print(f"   最早: {earliest['Hardware name']} ({earliest['release_year']:.0f}) - {earliest['int8_tops']:.1f} TOP/s")
print(f"   最新: {latest['Hardware name']} ({latest['release_year']:.0f}) - {latest['int8_tops']:.1f} TOP/s")
print(f"   增长倍数: {growth:.1f}x")
print(f"   CAGR: {cagr:.1f}%")

# Top 20 排行
df_int8_top20 = get_top_n(df_int8, 'int8_tops', n=20)
plot_barh(df_int8_top20, x_col='int8_tops', y_col='Hardware name',
          title="INT8 推理算力 Top 20", xlabel="INT8 算力 (TOP/s)", ylabel="硬件名称",
          filename="int8_top20_ranking.png", subdir='01_perf_trends')
save_table(df_int8_top20[['Hardware name', 'Manufacturer', 'release_year', 'int8_tops', 'TDP (W)']],
           'top20_int8_performance.csv')

print("✅ INT8 推理算力分析完成")


## Block 08 - FP16/BF16 训练算力演进（补充分析）

**目标**: 训练场景的算力趋势（FP16/BF16）


In [ ]:
# ============================================================
# FP16/BF16 训练算力演进分析（补充）
# ============================================================

print("\n" + "="*60)
print("📊 FP16/BF16 训练算力演进分析（补充）")
print("="*60)

df_fp16 = df[df['perf_fp16_tflops'].notna()].copy()
print(f"有 FP16/BF16 数据: {len(df_fp16)} / {len(df)} ({len(df_fp16)/len(df)*100:.1f}%)")

# FP16 训练算力趋势图
plot_scatter_trend(
    df_fp16, x_col='release_year', y_col='perf_fp16_tflops', hue_col='Manufacturer',
    title="FP16/BF16 训练算力演进 (2012-2025)\n对数轴展示指数增长",
    xlabel="发布年份", ylabel="FP16/BF16 算力 (TFLOP/s, log)",
    filename="fp16_performance_over_time.png", subdir='01_perf_trends',
    log_y=True, annotate_top_n=15, annotate_col='Hardware name'
)

# FP16 增长统计
df_fp16_sorted = df_fp16.sort_values('release_year')
if len(df_fp16_sorted) > 0:
    earliest_fp16 = df_fp16_sorted.iloc[0]
    latest_fp16 = df_fp16_sorted.iloc[-1]
    growth_fp16 = latest_fp16['perf_fp16_tflops'] / earliest_fp16['perf_fp16_tflops']
    years_fp16 = latest_fp16['release_year'] - earliest_fp16['release_year']
    cagr_fp16 = (growth_fp16 ** (1/years_fp16) - 1) * 100 if years_fp16 > 0 else 0
    
    print(f"\n📈 FP16/BF16 算力增长:")
    print(f"   最早: {earliest_fp16['Hardware name']} ({earliest_fp16['release_year']:.0f}) - {earliest_fp16['perf_fp16_tflops']:.1f} TFLOP/s")
    print(f"   最新: {latest_fp16['Hardware name']} ({latest_fp16['release_year']:.0f}) - {latest_fp16['perf_fp16_tflops']:.1f} TFLOP/s")
    print(f"   增长倍数: {growth_fp16:.1f}x")
    print(f"   CAGR: {cagr_fp16:.1f}%")

# FP16 Top 20 排行
df_fp16_top20 = get_top_n(df_fp16, 'perf_fp16_tflops', n=20)
plot_barh(df_fp16_top20, x_col='perf_fp16_tflops', y_col='Hardware name',
          title="FP16/BF16 训练算力 Top 20", xlabel="FP16/BF16 算力 (TFLOP/s)", ylabel="硬件名称",
          filename="fp16_top20_ranking.png", subdir='01_perf_trends')
save_table(df_fp16_top20[['Hardware name', 'Manufacturer', 'release_year', 'perf_fp16_tflops', 'TDP (W)']],
           'top20_fp16_performance.csv')

print("✅ FP16/BF16 训练算力分析完成")



## Block 09 - 按厂商分组的性能趋势对比

**目标**: 对比不同厂商的算力演进轨迹

In [ ]:

# ============================================================
# 按厂商分组的性能趋势对比
# ============================================================

print("\n" + "="*60)
print("📊 按厂商分组的性能趋势对比")
print("="*60)

# INT8 按厂商分组趋势
if len(df_int8) > 0:
    manufacturer_counts = df_int8['Manufacturer'].value_counts()
    major_manufacturers = manufacturer_counts[manufacturer_counts >= 3].index.tolist()
    df_int8_major = df_int8[df_int8['Manufacturer'].isin(major_manufacturers)].copy()
    
    if len(major_manufacturers) > 0:
        plot_facet_trend(
            df_int8_major, x_col='release_year', y_col='int8_tops', facet_col='Manufacturer',
            title="INT8 推理算力演进 - 按厂商分组对比",
            xlabel="发布年份", ylabel="INT8 算力 (TOP/s, log)",
            filename="int8_performance_by_manufacturer.png", subdir='01_perf_trends', log_y=True
        )
        print(f"✅ INT8 按厂商分组趋势图已生成（{len(major_manufacturers)} 个主要厂商）")

# FP16 按厂商分组趋势
if len(df_fp16) > 0:
    manufacturer_counts_fp16 = df_fp16['Manufacturer'].value_counts()
    major_manufacturers_fp16 = manufacturer_counts_fp16[manufacturer_counts_fp16 >= 3].index.tolist()
    df_fp16_major = df_fp16[df_fp16['Manufacturer'].isin(major_manufacturers_fp16)].copy()
    
    if len(major_manufacturers_fp16) > 0:
        plot_facet_trend(
            df_fp16_major, x_col='release_year', y_col='perf_fp16_tflops', facet_col='Manufacturer',
            title="FP16/BF16 训练算力演进 - 按厂商分组对比",
            xlabel="发布年份", ylabel="FP16/BF16 算力 (TFLOP/s, log)",
            filename="fp16_performance_by_manufacturer.png", subdir='01_perf_trends', log_y=True
        )
        print(f"✅ FP16/BF16 按厂商分组趋势图已生成（{len(major_manufacturers_fp16)} 个主要厂商）")

# 综合对比图（所有厂商在同一张图上，用颜色区分）
if len(df_int8) > 0:
    plot_scatter_trend(
        df_int8, x_col='release_year', y_col='int8_tops', hue_col='Manufacturer',
        title="INT8 推理算力演进 - 主要厂商对比\n（颜色区分厂商，对数轴）",
        xlabel="发布年份", ylabel="INT8 算力 (TOP/s, log)",
        filename="performance_by_manufacturer.png", subdir='01_perf_trends',
        log_y=True, annotate_top_n=10, annotate_col='Hardware name'
    )
    print("✅ 综合厂商对比图已生成（performance_by_manufacturer.png）")

print("✅ RQ1 分析完成（推理 + 训练 + 厂商对比）")


## Block 10 - RQ2: 能效对比

**目标**: 能效 Top 20 + 能效 vs 算力散点图

In [ ]:
# ============================================================
# RQ2: 能效对比分析
# ============================================================

print("\n" + "="*60)
print("📊 RQ2: 能效对比分析")
print("="*60)

df_eff = df[df['efficiency_int8_per_w'].notna()].copy()
print(f"有能效数据: {len(df_eff)} / {len(df)} ({len(df_eff)/len(df)*100:.1f}%)")

# 能效 Top 20
df_eff_top20 = get_top_n(df_eff, 'efficiency_int8_per_w', n=20)
plot_barh(df_eff_top20, x_col='efficiency_int8_per_w', y_col='Hardware name',
          title="INT8 推理能效 Top 20\n（TOP/s/W）", xlabel="能效 (TOP/s/W)", ylabel="硬件名称",
          filename="energy_efficiency_top20.png", subdir='02_efficiency')
save_table(df_eff_top20[['Hardware name', 'Manufacturer', 'int8_tops', 'TDP (W)', 'efficiency_int8_per_w']],
           'top20_energy_efficiency.csv')

# 能效 vs 算力散点图（增加标注，改进样式）
plot_scatter_trend(
    df_eff, x_col='int8_tops', y_col='efficiency_int8_per_w', hue_col='Manufacturer',
    title="能效 vs 算力\n右上角为\"甜点区\"（高算力+高能效）",
    xlabel="INT8 算力 (TOP/s, log)", ylabel="能效 (TOP/s/W)",
    filename="efficiency_vs_performance.png", subdir='02_efficiency',
    log_x=True, annotate_top_n=15, annotate_col='Hardware name'
)

# 统计能效提升
df_eff_sorted = df_eff.sort_values('release_year')
early_eff = df_eff_sorted.iloc[:10]['efficiency_int8_per_w'].mean()
recent_eff = df_eff_sorted.iloc[-10:]['efficiency_int8_per_w'].mean()
eff_improvement = (recent_eff / early_eff - 1) * 100

print(f"\n📈 能效提升:")
print(f"   早期平均（前 10）: {early_eff:.2e} TOP/s/W")
print(f"   近期平均（后 10）: {recent_eff:.2e} TOP/s/W")
print(f"   提升幅度: {eff_improvement:.1f}%")

print("\n📌 发现:")
print("- Google TPU v7 / NVIDIA H200 / AWS Trainium3 能效领先")
print("- 能效与算力不完全正相关（高算力≠高能效）")
print("- 能效提升主要来自制程进步（7nm→5nm）+ 架构优化")

print("✅ RQ2 分析完成")


## Block 11 - RQ3: 显存/带宽匹配

**目标**: 气泡图（x=带宽，y=算力，size=显存）


In [ ]:
# ============================================================
# RQ3: 显存/带宽匹配分析
# ============================================================

print("\n" + "="*60)
print("📊 RQ3: 显存/带宽匹配分析")
print("="*60)

df_mem = df[df['int8_tops'].notna() & df['mem_gb'].notna() & df['mem_bw_tbs'].notna()].copy()
print(f"有完整显存/带宽数据: {len(df_mem)} / {len(df)} ({len(df_mem)/len(df)*100:.1f}%)")

# 气泡图
plot_bubble(
    df_mem, x_col='mem_bw_tbs', y_col='int8_tops', size_col='mem_gb', hue_col='Manufacturer',
    title="算力 vs 带宽 气泡图\n气泡大小=显存容量（GB）",
    xlabel="显存带宽 (TB/s, log)", ylabel="INT8 算力 (TOP/s, log)",
    filename="compute_memory_bandwidth_bubble.png", subdir='03_memory_bandwidth',
    log_x=True, log_y=True, size_scale=5
)

# 算力-带宽比（识别瓶颈）
df_mem['compute_to_bw_ratio'] = df_mem['int8_tops'] / df_mem['mem_bw_tbs']
df_mem_sorted = df_mem.sort_values('compute_to_bw_ratio', ascending=False)

print(f"\n⚠️  算力-带宽比 Top 5（可能 memory-bound）:")
print(df_mem_sorted[['Hardware name', 'int8_tops', 'mem_bw_tbs', 'compute_to_bw_ratio']].head(5).to_string(index=False))

# 分布图
plot_distribution(df_mem, col='compute_to_bw_ratio',
                  title="算力-带宽比分布\n比值越高越可能 memory-bound",
                  xlabel="算力/带宽比", filename="compute_to_bandwidth_ratio_dist.png",
                  subdir='03_memory_bandwidth', log_scale=True)

print("\n📌 发现:")
print("- 大多数硬件\"算力-带宽\"同步增长")
print("- HBM3 是关键技术（H200/MI300X 都用 HBM3）")
print("- 部分高算力硬件存在带宽瓶颈")

print("✅ RQ3 分析完成")



## Block 12 - RQ4: 价格分析（限有价格硬件）

**目标**: 性价比 Top 10（⚠️ 仅限消费级 GPU）

In [ ]:

# ============================================================
# RQ4: 价格与性价比分析
# ============================================================

print("\n" + "="*60)
print("📊 RQ4: 价格与性价比分析")
print("="*60)

df_price = df[df['Release price (USD)'].notna() & df['int8_tops'].notna()].copy()
print(f"有价格数据: {len(df_price)} / {len(df)} ({len(df_price)/len(df)*100:.1f}%)")

if len(df_price) > 0:
    df_price['price_performance'] = df_price['int8_tops'] / df_price['Release price (USD)']
    
    # 价格 vs 性能
    plot_scatter_trend(
        df_price, x_col='Release price (USD)', y_col='int8_tops', hue_col='Manufacturer',
        title="价格 vs INT8 算力\n⚠️ 仅限有价格硬件（消费级 GPU）",
        xlabel="发布价格 (USD)", ylabel="INT8 算力 (TOP/s)",
        filename="price_vs_performance.png", subdir='04_price_value',
        annotate_top_n=5, annotate_col='Hardware name'
    )
    
    # 性价比 Top 10
    df_price_top10 = get_top_n(df_price, 'price_performance', n=10)
    plot_barh(df_price_top10, x_col='price_performance', y_col='Hardware name',
              title="INT8 性价比 Top 10\nTOP/s per USD",
              xlabel="性价比 (TOP/s per USD)", ylabel="硬件名称",
              filename="price_performance_top10.png", subdir='04_price_value')
    save_table(df_price_top10[['Hardware name', 'Manufacturer', 'Release price (USD)', 'int8_tops', 'price_performance']],
               'top10_price_performance.csv')
    
    print("\n📌 发现（⚠️ 仅限有价格硬件）:")
    print("- 中端 GPU（RTX 4070/4080）性价比较高")
    print("- 旗舰不一定划算（性能翻倍，价格翻 3-4 倍）")
    print("- 数据中心硬件无公开价格，需联系厂商")
else:
    print("⚠️  无有效价格数据")

print("✅ RQ4 分析完成")


## Block 13 - 总结与选型建议

**目标**: 综合结论 + 硬件推荐

In [ ]:
# ============================================================
# 总结与选型建议
# ============================================================

print("\n" + "="*60)
print("📊 总结与选型建议")
print("="*60)

print("\n### 核心发现:")
print("1. INT8 算力 2012-2025 增长 {:.1f}x，CAGR {:.1f}%".format(growth, cagr))
print("2. Google TPU v7 / NVIDIA H200 / AWS Trainium3 能效领先")
print("3. 大多数硬件\"算力-带宽\"同步增长，HBM3 是关键")
print("4. 价格数据缺失严重（~80%），仅限消费级 GPU 可对比")

print("\n### 🎯 硬件选型建议:")
print("\n**场景一：大模型训练**")
print("   推荐：H200 SXM (141GB) / H100 SXM (80GB) / MI300X")
print("   理由：FP16/BF16 算力强 + 大显存 + 高带宽")

print("\n**场景二：大模型推理**")
print("   推荐：TPU v7 / Trainium3 / H200 / L40S")
print("   理由：INT8/FP8 算力高 + 能效优异")

print("\n**场景三：中小模型（预算有限）**")
print("   推荐：RTX 4080 / 4070 Ti / 3090")
print("   理由：性价比高 + 显存够用（12-24GB）")

print("\n⚠️  注意事项:")
print("- 数据中心硬件无公开价格，需联系厂商询价")
print("- 能效长期收益：计算 3-5 年 TCO（总拥有成本）")
print("- 软件兼容性：TPU/Trainium 需特定框架（JAX/Neuron SDK）")

print("\n✅ 所有分析完成！")
print(f"📁 图表已保存至: {VIZ_DIR}")
print(f"📁 表格已保存至: {OUTPUT_TABLES_DIR}")
print("="*60)
